# Booz Allen Spring 2025 Codefest: Challenge 1 
### Isaiah Byrd, Kyler Gelissen, David Ameh

In [2]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
print(path)

dataset_root = os.path.join(path, "garbage-dataset")

C:\Users\Kyler\.cache\kagglehub\datasets\sumn2u\garbage-classification-v2\versions\8


In [ ]:
import os

# Create a directory to store the dataset
dataset_dir = "garbage_classification_dataset"
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    print(f"Directory {dataset_dir} created.")
else:
    pass
    print(f"Directory {dataset_dir} already exists.")

Directory garbage_classification_dataset already exists.


In [ ]:
import subprocess
import sys

# Function to install a package
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    "matplotlib",
    "pandas",
    "numpy",
    "torch",
    "torchvision",
    "opencv-python",
    "kagglehub",
    "scikit-learn"
]

# Install each package if not already installed
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

# Importing the libraries
import matplotlib.pyplot as plt
import copy
import os
import pandas as pd
import numpy as np
import torch as th
import json
import random
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset
from torchvision.models import ResNet50_Weights
from sklearn.model_selection import StratifiedShuffleSplit

c:\Users\Kyler\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(f"Using device: {device}")


#device = th.device("cuda" if th.cuda.is_available() else "cpu")
#print(f"Using device: {device}")

#Change the memory fraction to limit the GPU memory usage
#Set the memory fraction of the total GPU memory
memory_fraction = 0.80

if(device.type == "cuda"):
    print("GPU is available")
    th.cuda.set_memory_fraction(memory_fraction, device=device.index)

Using device: cpu


In [6]:
#Defines the data transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), #Image needs to be resized to 224x224 for ResNet requirement
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #Standard normalization for ResNet
])

#Load the dataset
complete_dataset = datasets.ImageFolder(root=dataset_root, transform=data_transforms) 
dataset_size = len(complete_dataset)

In [7]:
#We are going to do a 80-10-10 split of the dataset to start
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size
print(f"Dataset size: {dataset_size}")
print(f"Train size: {train_size}")
print(f"Validation size: {val_size}")
print(f"Test size: {test_size}")

#Using random_split to split the dataset into train, validation and test sets
train_dataset, val_dataset, test_dataset = th.utils.data.random_split(complete_dataset, [train_size, val_size, test_size])

Dataset size: 19762
Train size: 15809
Validation size: 1976
Test size: 1977


In [ ]:
targets = np.array([sample[1] for sample in complete_dataset.samples])
StratShuffleSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) #Still doing 80 - 10 10 (20)
train_idx, val_test_index = next(StratShuffleSplit.split(np.zeros(dataset_size), targets))
StratShuffleSplit_val_test = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42) #This splits the 20 into 10 and 10
val_idx, test_idx = next(StratShuffleSplit_val_test.split(np.zeros(len(val_test_index)), targets[val_test_index]))

train_dataset = th.utils.data.Subset(complete_dataset, train_idx)
val_dataset = th.utils.data.Subset(complete_dataset, val_idx)
test_dataset = th.utils.data.Subset(complete_dataset, test_idx)

In [ ]:
#Creating the dataloaders
batch_size = 64 #Update this if memory permits
num_workers = 4 #Decrease if colab is mad
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True) 
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

#Fixed the dataset classes so it doesn't need to be manually set
class_names = complete_dataset.classes
num_classes = len(class_names)
print(f"Class names from dataset: {class_names}")
print(f"Number of classes: {len(class_names)}")

Class names from dataset: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Number of classes: 10


In [ ]:
model = models.resnet50(weights=ResNet50_Weights.DEFAULT) #Using ResNet50 as the base model for transfer learning

#This takes the last layer of the model and replaces it with a new one
num_ftrs = model.fc.in_features 
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device) #Moving the model to the device

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\Kyler/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 25.9MB/s]


In [13]:
criterion = nn.CrossEntropyLoss()
#Learning rate is set to 1e-4 as a starting point, update this if needed
learning_rate = 1.e-4 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
max_epochs = 50 #Updated if needed
patience = 3 #Early stopping patience
best_val_acc = 0.0
epochs_without_improvement = 0
best_model_wts = copy.deepcopy(model.state_dict())
train_accuracy_history = []
val_accuracy_history = []

In [ ]:
#Main training loop
for epoch in  range(max_epochs):
    model.train() #Sets the model to training mode
    running_corrects, running_samples = 0, 0

    for inputs, labels in train_loader:
        #Move the inputs and labels to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        #Zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward() #Backpropagation step
        optimizer.step() #Optimizer step

        #Get the stupid predictions
        preds = th.argmax(outputs, dim=1)
        preds = preds.cpu().numpy()
        labels = labels.cpu().numpy()
        running_corrects += np.sum(preds == labels)
        running_samples += labels.shape[0]

    #Calculate the accuracy
    epoch_train_accuracy = running_corrects / running_samples
    train_accuracy_history.append(epoch_train_accuracy)

    #Validation step
    model.eval() #Sets the model to evaluation mode
    running_corrects = 0
    running_samples = 0
    with th.no_grad(): #Disables gradient calculation
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            preds = th.argmax(outputs, dim=1)
            preds = preds.cpu().numpy()
            labels = labels.cpu().numpy()
            running_corrects += np.sum(preds == labels)
            running_samples += labels.shape[0]

    #Calculate the accuracy
    epoch_val_accuracy = running_corrects / running_samples
    val_accuracy_history.append(epoch_val_accuracy)

    print(f"Epoch {epoch+1}/{max_epochs} - Train accuracy: {epoch_train_accuracy:.4f} - Validation accuracy: {epoch_val_accuracy:.4f}")

    #Early stopping block
    if epoch_val_accuracy  > best_val_acc:
        best_val_acc = epoch_val_accuracy
        epochs_without_improvement = 0
        best_model_wts = copy.deepcopy(model.state_dict())
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch+1} due to no improvement in validation accuracy for {patience} epochs.")
            break

train_accuracy_history = np.array(train_accuracy_history)
val_accuracy_history = np.array(val_accuracy_history)

#Saves it
model.load_state_dict(best_model_wts)

In [ ]:
#Plotting the training and validation accuracy
plt.figure(figsize=(8, 5))

epochs = np.arange(1, len(train_accuracy_history) + 1)
plt.plot(epochs, train_accuracy_history, label='Train Accuracy')
plt.plot(epochs, val_accuracy_history, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
#Test loop
model.eval()
running_corrects = 0
running_samples = 0
with th.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        preds = th.argmax(outputs, dim=1)
        running_corrects += th.sum(preds == labels).item()  #More efficient on GPU
        running_samples += labels.size(0)
test_acc = running_corrects / running_samples
print(f"Test Accuracy (full set): {test_acc:.4f}")

In [ ]:
#Saves the model
model_path = "garbage_classification_model.pth"
th.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
#Function to display a batch of images
with th.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        preds = th.argmax(outputs, dim=1)
        images = inputs.cpu().numpy()  #Shape:(batch_size, 3, 224, 224)
        actual_labels = labels.cpu().numpy()
        predicted_labels = preds.cpu().numpy()
        break  #One batch for visualization

In [ ]:
#We need to denormalize the images to display them
def denormalize(image):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    #This rescales the image to the original range
    image = image * std[:, None, None] + mean[:, None, None]
    return image

In [ ]:
indices = random.sample(range(len(images)), 6)
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i, idx in enumerate(indices):
    image = images[idx]  #Shape: (3, 224, 224)
    actual = class_names[actual_labels[idx]]
    predicted = class_names[predicted_labels[idx]]
    
    #Denormalize and transpose image to (H, W, C) for Matplotlib
    image = denormalize(image)
    image = np.transpose(image, (1, 2, 0))
    image = np.clip(image, 0, 1)  #Ensure pixel values are between 0 and 1
    
    #Display the image
    axes[i].imshow(image)
    
    #Set title with actual and predicted (green if correct, red if incorrect)
    title_color = 'green' if actual == predicted else 'red'
    axes[i].set_title(f"Actual: {actual}\nPredicted: {predicted}", 
                      color=title_color, 
                      fontsize=10)
    axes[i].axis('off')

#Adjust layout and display the plot
plt.tight_layout()
plt.show()

### GUI for uploading an image to the model:

In [6]:
import tkinter as tk
from tkinter import messagebox
from tkinter.ttk import Button
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageTk, UnidentifiedImageError

CANV_WIDTH, CANV_HEIGHT = 600, 500

class ImageUploader:
    def __init__(self, master=None):
        """
        Initialize the ImageUploader class.
        
        If master is None, it creates a new Tkinter window.
        Otherwise, it uses the provided master window (for embedding in another program).
        """
        self.master = master or tk.Tk()
        self.master.title("Image Uploader")
        self.master.geometry(f"{600}x{550}")

        self.canvas = tk.Canvas(self.master, width=CANV_WIDTH, height=CANV_HEIGHT, bg="white")
        self.canvas.pack()

        self.btn = Button(self.master, text="Upload Image", command=self.upload_image)
        self.btn.pack()

        if master is None:  # Only run mainloop if this is the main script
            self.master.mainloop()

    def upload_image(self):
        """Handles the image upload and display process."""
        try:
            img_path = askopenfilename()  # Open file dialog to select an image
            if not img_path:
                return  # If no file is selected, exit function

            im = Image.open(img_path)  # Open image using PIL
            img_width, img_height = im.size  # Get image dimensions

            if img_width > 224 or img_height > 224:  # Resize image if too large
                while img_width > CANV_WIDTH or img_height > CANV_HEIGHT:
                    img_width *= 0.99
                    img_height *= 0.99

                im = im.resize((int(img_width), int(img_height)))
                messagebox.showinfo("Image Resized", "The image has been resized to fit the model")

            img = ImageTk.PhotoImage(im)  # Convert image to Tkinter format
            self.canvas.image = img  # Keep reference to image
            self.canvas.create_image(CANV_WIDTH/2, CANV_HEIGHT/2, image=img, anchor=tk.CENTER)  # Display image on canvas

        except UnidentifiedImageError:
            messagebox.showinfo("Error", "Invalid file type")

In [ ]:
import torch as th
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
#from google.colab import files  # For file uploads in Colab

# 1) Define device and load model
device = th.device("cuda" if th.cuda.is_available() else "cpu")

class_names = complete_dataset.classes
num_classes = len(class_names)

model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model.load_state_dict(th.load("garbage_classification_model.pth", map_location=device))
model.to(device)
model.eval()


data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def predict_image(pil_image):
    #Move to device
    image_tensor = data_transforms(pil_image).unsqueeze(0).to(device)
    with th.no_grad():
        output = model(image_tensor)
        _, predicted = th.max(output, 1)
    return class_names[predicted.item()]


uploaded = files.upload() 
img_path = list(uploaded.keys())[0]  

# 5) Open it, predict, and display
uploaded_image = Image.open(img_path).convert("RGB")
predicted_class = predict_image(uploaded_image)
print(f"Predicted class: {predicted_class}")

plt.imshow(np.array(uploaded_image))
plt.title(f"Predicted: {predicted_class}")
plt.axis('off')
plt.show()


c:\Users\Kyler\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Kyler\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.